In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)

In [3]:
#REDUCE MEMORY USAGE
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train_df = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset/train.csv")
test_df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset/test.csv")
building_metadata_df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset/building_metadata.csv")
weather_test_df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset/weather_test.csv")
weather_train_df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset/weather_train.csv")

In [5]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
building_metadata_df = reduce_mem_usage(building_metadata_df)
weather_test_df = reduce_mem_usage(weather_test_df)
weather_train_df = reduce_mem_usage(weather_train_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)


In [6]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [7]:
type(train_df["timestamp"][0])

str

In [8]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.000000,6.0,20.00000,NaN,1019.5,0.0,0.000000
1,0,2016-01-01 01:00:00,24.406250,NaN,21.09375,-1.0,1020.0,70.0,1.500000
2,0,2016-01-01 02:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000
3,0,2016-01-01 03:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609


In [9]:
building_metadata_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [10]:
max(train_df["meter_reading"])

21904700.0

In [11]:
train_combined = pd.merge(train_df,building_metadata_df,on='building_id',how='left')
test_combined  = pd.merge(test_df,building_metadata_df,on='building_id',how='left')


In [12]:
train_combined = pd.merge(train_combined,weather_train_df,on=['site_id','timestamp'],how='left')
test_combined  = pd.merge(test_combined,weather_test_df,on=['site_id','timestamp'],how='left')

In [13]:
temp1 = pd.to_datetime(test_combined["timestamp"])
day = temp1.dt.day
day_of_week = temp1.dt.dayofweek
month = temp1.dt.month
year = temp1.dt.year
hour = temp1.dt.hour
minute = temp1.dt.minute
second = temp1.dt.second

In [14]:
test_combined["day"] = day
test_combined["day_of_week"] = day_of_week
test_combined["month"] = month
test_combined["year"] = year
test_combined["hour"] = hour
test_combined["minute"] = minute
test_combined["second"] = second

In [15]:
test_combined.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,day_of_week,month,year,hour,minute,second
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,6,1,2017,0,0,0
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,6,1,2017,0,0,0
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,6,1,2017,0,0,0
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,6,1,2017,0,0,0
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609,1,6,1,2017,0,0,0


In [16]:
train_combined.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [17]:
temp = pd.to_datetime(train_combined['timestamp'])


In [18]:
#temp1 = []
#for i in temp:
#    ts = pd.Timestamp(i)
#    ts.to_pydatetime()
#    temp1.append(ts)
#
#
#train_combined['timestamp'] = temp1

In [19]:
train_combined.shape

(20216100, 16)

In [20]:
del train_df
del test_df 
del building_metadata_df
del weather_test_df
del weather_train_df 


In [21]:
import gc 
gc.collect()

100

In [22]:
day = temp.dt.day
day_of_week = temp.dt.dayofweek
month = temp.dt.month
year = temp.dt.year
hour = temp.dt.hour
#minute = temp.dt.minute
#second = temp.dt.second


In [23]:
train_combined["day"] = day
train_combined["day_of_week"] = day_of_week
train_combined["month"] = month
train_combined["year"] = year
train_combined["hour"] = hour
#train_combined["minute"] = minute
#train_combined["second"] = second

In [24]:
del day
del day_of_week
del month
del year
del hour
del minute
del second
gc.collect()

60

In [25]:
train_combined.shape

(20216100, 21)

In [26]:
train_combined = train_combined.drop("timestamp",axis = 1)

In [27]:
train_combined["floor_count"].nunique()

18

In [28]:
temp = train_combined.isnull().any()
for count,i in enumerate(temp):
    if i== True:
        print(train_combined.columns.values[count])
        print((train_combined[train_combined.columns.values[count]].isnull().sum())/train_combined.shape[0])

year_built
0.5999003269671203
floor_count
0.826527718006935
air_temperature
0.004781238715677109
cloud_coverage
0.43655131306236117
dew_temperature
0.00495347767373529
precip_depth_1_hr
0.18544739094088375
sea_level_pressure
0.06092515371411895
wind_direction
0.0716779200736047
wind_speed
0.007107008770237583


In [29]:
train_combined.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,day_of_week,month,year,hour
0,0,0,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,1,4,1,2016,0
1,1,0,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,1,4,1,2016,0
2,2,0,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,1,4,1,2016,0
3,3,0,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,1,4,1,2016,0
4,4,0,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,1,4,1,2016,0


In [30]:
train_combined["air_temperature"] = train_combined["air_temperature"].astype(float)
print(train_combined["air_temperature"].mean())

15.987972114626087


In [31]:
train_combined = train_combined.drop("floor_count",axis = 1)

In [32]:
train_combined = train_combined.drop("building_id",axis = 1)

In [33]:
y = train_combined["meter_reading"]
train_combined = train_combined.drop("meter_reading",axis = 1)

In [34]:
 temp = train_combined["year_built"].mode()

In [35]:
train_combined["year_built"] = train_combined["year_built"].fillna(int(temp))
train_combined["air_temperature"] = train_combined["air_temperature"].astype(float)
train_combined["air_temperature"] = train_combined["air_temperature"].fillna(int(train_combined["air_temperature"].mean()))
train_combined["cloud_coverage"] = train_combined["cloud_coverage"].fillna(int(train_combined["air_temperature"].mode()))
train_combined["dew_temperature"] = train_combined["dew_temperature"].astype(float)
train_combined["dew_temperature"] = train_combined["dew_temperature"].fillna(int(train_combined["dew_temperature"].mean()))
train_combined["precip_depth_1_hr"] = train_combined["precip_depth_1_hr"].astype(float)
train_combined["precip_depth_1_hr"] = train_combined["precip_depth_1_hr"].fillna(int(train_combined["precip_depth_1_hr"].mode()))
train_combined["sea_level_pressure"] = train_combined["sea_level_pressure"].astype(float)
train_combined["sea_level_pressure"] = train_combined["sea_level_pressure"].fillna(int(train_combined["sea_level_pressure"].mean()))
train_combined["wind_direction"] = train_combined["wind_direction"].astype(float)
train_combined["wind_direction"] = train_combined["wind_direction"].fillna(int(train_combined["wind_direction"].mean()))
train_combined["wind_speed"] = train_combined["wind_speed"].astype(float)
train_combined["wind_speed"] = train_combined["wind_speed"].fillna(int(train_combined["wind_speed"].mean()))


In [36]:
train_combined.isnull().any().any()

False

In [37]:
train_combined.head()

,meter,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,day_of_week,month,year,hour
0,0,0,Education,7432,2008.0,25.0,6.0,20.0,0.0,1019.5,0.0,0.0,1,4,1,2016,0
1,0,0,Education,2720,2004.0,25.0,6.0,20.0,0.0,1019.5,0.0,0.0,1,4,1,2016,0
2,0,0,Education,5376,1991.0,25.0,6.0,20.0,0.0,1019.5,0.0,0.0,1,4,1,2016,0
3,0,0,Education,23685,2002.0,25.0,6.0,20.0,0.0,1019.5,0.0,0.0,1,4,1,2016,0
4,0,0,Education,116607,1975.0,25.0,6.0,20.0,0.0,1019.5,0.0,0.0,1,4,1,2016,0


In [38]:
for i in train_combined.columns:
    print(i)
    print(train_combined[i].nunique())

meter
4
site_id
16
primary_use
16
square_feet
1397
year_built
116
air_temperature
619
cloud_coverage
11
dew_temperature
522
precip_depth_1_hr
128
sea_level_pressure
133
wind_direction
44
wind_speed
58
day
31
day_of_week
7
month
12
year
1
hour
24


In [39]:
train_combined["cloud_coverage"].unique()

array([ 6., 15.,  8.,  4.,  0.,  2.,  7.,  5.,  3.,  9.,  1.])

In [40]:
# Here we are treating time as a category as the values themselves do not have any meaning
# although we can also treat them as a numeric variable
# I should try that too and maybe compare the performance of both in both tree and non tree based models.


# THIS IS VERRRRY MEMORY INTENSIVE
train_combined["meter"] = train_combined["meter"].astype("category")
train_combined["site_id"] = train_combined["site_id"].astype("category")
train_combined["primary_use"] = train_combined["primary_use"].astype("category")
train_combined["day_of_week"] = train_combined["day_of_week"].astype("category")
train_combined["month"] = train_combined["month"].astype("category")
train_combined["hour"] = train_combined["hour"].astype("category")
train_combined["day"] = train_combined["day"].astype("category")

In [41]:
train_combined = reduce_mem_usage(train_combined)

Mem. usage decreased to 713.35 Mb (53.2% reduction)


In [42]:
dummy = pd.get_dummies(train_combined)

In [43]:
dummy = reduce_mem_usage(dummy)

Mem. usage decreased to 2699.14 Mb (0.0% reduction)


In [44]:
del train_combined
gc.collect()

30

In [45]:
# NOW ALL THE NUL VALUES HAVE BEEN REMOVED
# Now scaling should be done

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(dummy)
normal = scaler.transform(dummy)

In [46]:
del dummy
gc.collect()

20

In [47]:
normal.shape

(20216100, 120)

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normal, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(X_train,y_train)

C:\Users\ishtd\Anaconda3\envs\Vision\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
